<a href="https://colab.research.google.com/github/Sarvagya4/LangChain/blob/main/Langchain6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain==0.1.4
!pip install -q transformers==4.37.1
!pip install -q accelerate==0.26.1

In [ ]:
!pip install huggingface_hub==0.20.2

In [ ]:
from getpass import getpass
hf_key = getpass("Hugging Face Key: ")


In [ ]:
!huggingface-cli login --token $hf_key

In [ ]:
from langchain import PromptTemplate
#from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

from langchain.llms import HuggingFacePipeline
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [ ]:
import torch
import os
import numpy as np
from torch import cuda, bfloat16

In [ ]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [ ]:
print(device)

In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_key
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
    use_auth_token=hf_key
)
model.eval()
print(f"Model loaded on {device}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          use_aut_token=hf_key)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.1,
    #do_sample=False,
    top_p=0,
    #trust_remote_code=True,
    repetition_penalty=1.1,
    return_full_text=True,
    device_map='auto'
)

assistant_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
assistant_template = """
[INST]<<SYS>>You are {sentiment} assistant that responds to user comments,
using similar vocabulary than the user.
Stop answering text after answer the first user.<</SYS>>

User comment:{customer_request}[/INST]
assistant_response:
"""

In [ ]:
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

In [ ]:
output_parser = StrOutputParser()
assistant_chain = assistant_prompt_template | assistant_llm | output_parser

In [ ]:
def create_dialog(customer_request, sentiment):
    #calling the .invoke method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

In [ ]:
customer_request = """Your product is a piece of shit. I want my money back!"""

In [ ]:
assistant_response=create_dialog(customer_request, "nice")
print(assistant_response)

In [ ]:
assistant_response = create_dialog(customer_request, "most rude possible assistant")
print(assistant_response)

In [ ]:
moderator_template = """
[INST]<<SYS>>You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive an original comment and if it is impolite you must transform into polite.
Try to mantain the meaning when possible.<</SYS>>

Original comment: {comment_to_moderate}/[INST]
"""

# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template
)

In [ ]:
moderator_llm = assistant_llm

In [ ]:
moderator_chain = moderator_prompt_template | moderator_llm | output_parser

In [ ]:
moderator_says = moderator_chain.invoke({"comment_to_moderate": assistant_response})

print(moderator_says)

In [ ]:
assistant_moderated_chain = (
    {"comment_to_moderate":assistant_chain}
    |moderator_chain
)

In [ ]:
from langchain.callbacks.tracers import ConsoleCallbackHandler
assistant_moderated_chain.invoke({"sentiment": "really rude", "customer_request": customer_request},
                                 config={'callbacks':[ConsoleCallbackHandler()]})